## Finetuning Dataset

In [3]:
import pandas as pd

df = pd.read_parquet("hf://datasets/iamtarun/python_code_instructions_18k_alpaca/data/train-00000-of-00001-8b6e212f3e1ece96.parquet")

In [4]:
df.head()

,instruction,input,output,prompt
0,Create a function to calculate the sum of a se...,"[1, 2, 3, 4, 5]",# Python code\ndef sum_sequence(sequence):\n ...,Below is an instruction that describes a task....
1,Generate a Python code for crawling a website ...,website: www.example.com \ndata to crawl: phon...,import requests\nimport re\n\ndef crawl_websit...,Below is an instruction that describes a task....
2,Create a Python list comprehension to get the ...,,"[x*x for x in [1, 2, 3, 5, 8, 13]]",Below is an instruction that describes a task....
3,Generate a python script to perform this action.,"Given a string, remove all the consecutive dup...",def remove_duplicates(string): \n result = ...,Below is an instruction that describes a task....
4,Write a python script to generates random numb...,,def generate_random_divisible_number():\n i...,Below is an instruction that describes a task....


In [10]:
row = df.loc[0]

print("Instruction :\n", row.instruction, "\n", sep="")
print("Input :\n", row.input, "\n", sep="")
print("Output :\n", row.output, sep="")

Instruction :
Create a function to calculate the sum of a sequence of integers.

Input :
[1, 2, 3, 4, 5]

Output :
# Python code
def sum_sequence(sequence):
  sum = 0
  for num in sequence:
    sum += num
  return sum


In [11]:
print(row.prompt)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Create a function to calculate the sum of a sequence of integers.

### Input:
[1, 2, 3, 4, 5]

### Output:
# Python code
def sum_sequence(sequence):
  sum = 0
  for num in sequence:
    sum += num
  return sum


In [14]:
df.to_csv("./finetune-original.csv", index=False)

## Pre-training

In [1]:
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm

ds = load_dataset("codeparrot/codeparrot-clean")

Resolving data files:   0%|          | 0/54 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/108 [00:00<?, ?it/s]

In [5]:
print(data[0]["content"])

###############################################################################
##
##  Copyright (C) 2013-2014 Tavendo GmbH
##
##  Licensed under the Apache License, Version 2.0 (the "License");
##  you may not use this file except in compliance with the License.
##  You may obtain a copy of the License at
##
##      http://www.apache.org/licenses/LICENSE-2.0
##
##  Unless required by applicable law or agreed to in writing, software
##  distributed under the License is distributed on an "AS IS" BASIS,
##  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
##  See the License for the specific language governing permissions and
##  limitations under the License.
##
###############################################################################

from autobahn.asyncio.websocket import WebSocketClientProtocol, \
                                       WebSocketClientFactory

import asyncio



class MyClientProtocol(WebSocketClientProtocol):

   def onConnect(self, respo

In [2]:
data = []

for row in tqdm(ds["train"], desc="No. of Python codes", unit="iteration"):
    data.append(row)

No. of Python codes:   2%|▏         | 89273/5361373 [00:05<05:51, 15010.51iteration/s]


KeyboardInterrupt: 

In [8]:
df_ = pd.DataFrame(data)

# Default way to store CSV is raising some error with file seperators
# df_.to_csv("./pretrain-original.csv", index=False)

df_.head()

,repo_name,path,copies,size,content,license,hash,line_mean,line_max,alpha_frac,autogenerated
0,ahmedbodi/AutobahnPython,examples/asyncio/websocket/echo/client_corouti...,13,2044,##############################################...,apache-2.0,7822061744094950801,31.444444,79,0.623288,False
1,ifduyue/django,django/core/checks/registry.py,13,3108,from itertools import chain\n\nfrom django.uti...,bsd-3-clause,-2035686896372967697,30.714286,91,0.602317,False
2,kmike/scikit-learn,sklearn/utils/__init__.py,3,10094,"""""""\nThe :mod:`sklearn.utils` module includes ...",bsd-3-clause,2334709577611160651,26.883978,79,0.568060,False
3,houlixin/BBB-TISDK,linux-devkit/sysroots/i686-arago-linux/usr/lib...,593,13942,""""""" Python Character Mapping Codec cp1250 gene...",gpl-2.0,-6356832018515182181,44.413681,119,0.550351,False
4,dataxu/ansible,lib/ansible/modules/system/kernel_blacklist.py,125,4009,#!/usr/bin/python\n# encoding: utf-8 -*-\n\n# ...,gpl-3.0,8498771084445726761,24.864516,92,0.575206,False


Just to make sure all file are `.py`

In [14]:
file_types = df_["path"].map(lambda x: x.split(".")[-1])
file_types.unique()

array(['py'], dtype=object)

In [36]:
df_["size"].map(lambda x: float(x)).describe()

count    5.361373e+06
mean     1.005764e+04
std      2.251043e+04
min      1.024000e+03
25%      2.169000e+03
50%      4.328000e+03
75%      9.721000e+03
max      1.048355e+06
Name: size, dtype: float64

In [69]:
df_["size"] = df_["size"].map(lambda x: int(x))

small_pretrain_data = df_[df_["size"] <= 1500].reset_index(drop=True)
small_pretrain_data.head()

,repo_name,path,copies,size,content,license,hash,line_mean,line_max,alpha_frac,autogenerated
0,jdemel/gnuradio,gr-utils/modtool/templates/gr-newmod/docs/doxy...,3,1295,"#\n# Copyright 2010 Free Software Foundation, ...",gpl-3.0,-4949053946127237580,27.152174,100,0.642471,False
1,savi-dev/keystone,keystone/common/kvs.py,4,1477,# vim: tabstop=4 shiftwidth=4 softtabstop=4\n\...,apache-2.0,1685284111099958652,27.403846,75,0.631009,False
2,c1728p9/pyOCD,pyOCD/target/target_stm32f103rc.py,4,1457,"""""""\n mbed CMSIS-DAP debugger\n Copyright (c) ...",apache-2.0,-6004540841488146735,32.883721,151,0.728895,False
3,d40223223/2015cdbg6team0622,static/Brython3.1.1-20150328-091302/Lib/import...,608,1396,from javascript import JSObject\nfrom browser ...,gpl-3.0,834423359024821850,29.347826,70,0.537966,False
4,dharmabumstead/ansible,lib/ansible/plugins/action/command.py,117,1121,"# Copyright: (c) 2017, Ansible Project\n# GNU ...",gpl-3.0,6676477658934152115,36.366667,103,0.683318,False


In [71]:
data = []

for i in range(len(small_pretrain_data)):
    content = small_pretrain_data.loc[i, "content"]

    if "copyright" not in content.lower():
        data.append({
            "content": content
        })

len(data)

489463

In [73]:
small_pretrain_data = pd.DataFrame(data)
small_pretrain_data.to_csv("./pretrain-small.csv", index=False)

In [78]:
print(small_pretrain_data.loc[0].content)

from javascript import JSObject
from browser import window
import urllib.request

class TempMod:
  def __init__(self, name):
      self.name=name

#define my custom import hook (just to see if it get called etc).
class BaseHook:
  def __init__(self, fullname=None, path=None):
      self._fullname=fullname
      self._path=path    # we don't are about this...
      self._modpath=''
      self._module=''

  def find_module(self, name=None, path=None):
      if name is None:
         name=self._fullname

      for _i in ('libs/%s.js' % name, 'Lib/%s.py' % name, 
                 'Lib/%s/__init__.py' % name):
          _path="%s%s" % (__BRYTHON__.brython_path, _i)
          try:
            _fp,_,_headers=urllib.request.urlopen(_path)
            if _headers['status'] != 200:
               continue 
            self._module=_fp.read()
            self._modpath=_path
            return self
          except urllib.error.HTTPError as e:
            print(str(e))
            self._modpath=''